In [ ]:
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, ChebConv, SAGEConv
from torch_geometric.nn import global_mean_pool
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GraphConv 
from torch_geometric.nn import global_mean_pool
from Lib import *

class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, num_node_features, num_classes, seed):
        super(GCN, self).__init__()
        torch.manual_seed(seed)
        self.conv1 = GraphConv(num_node_features, hidden_channels)
        self.conv2 = GraphConv(hidden_channels, hidden_channels)
        self.conv3 = GraphConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels, num_classes)

    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        # 2. Readout layer
        x = global_mean_pool(x, batch)  # [batch_size, hidden_channels]
        # 3. Apply a final classifier
        x = F.dropout(x, p=0.5, training=self.training)
        x = self.lin(x)
        return x


def run(name_label, epochs, print_it, config):
    dataset= load_data(name_label)
    print(len(dataset))
    memory_train=[]
    memory_test=[]
    model = GCN(hidden_channels=64,\
        num_node_features= dataset.num_features,\
        num_classes=dataset.num_classes).to(config['device'])
    print(model)
    criterion = torch.nn.CrossEntropyLoss(reduction='none')
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
    #import torch.optim.lr_scheduler as lrs
    #scheduler = lrs.ExponentialLR(optimizer, gamma=0.9)
    accuracies_mem = []
    accuracies_one=[]
    Total_loss=[]
    Gen_loss=[]
    For_loss=[]
    n_Tasks=dataset.num_classes
    for i in range(n_Tasks):
        print("The task number", i)
        train_loader, test_loader, mem_train_loader, mem_test_loader,\
            memory_train, memory_test = continuum_Graph_classification(dataset, memory_train, memory_test, batch_size=64, task_id=i)
        for epoch in range(1,epochs):
            Total,Gen,For=train_CL(model, criterion, optimizer, mem_train_loader, train_loader, task=i, graph=1, node=0, params=config)
            Total_loss.append(Total)
            Gen_loss.append(Gen)
            For_loss.append(For)
            # print(epoch, print_it)
            if epoch%print_it==0:
                # scheduler.step()
                train_acc = test_GC(model, train_loader)
                test_acc = test_GC(model, test_loader)
                mem_train_acc = test_GC(model, mem_train_loader)
                mem_test_acc = test_GC(model, mem_test_loader)
                accuracies_mem.append(mem_test_acc)
                accuracies_one.append(test_acc)
        print("#########################################################################")
        print(f'Epoch: {epoch:03d}, Train Acc: {train_acc:.4f}, Test Acc: {test_acc:.4f}')
        print(f'Mem Train Acc: {mem_train_acc:.4f}, Mem Test Acc: {mem_test_acc:.4f}')
        print("#########################################################################")

    del model, criterion, optimizer, memory_train, memory_test, Total_loss, Gen_loss, For_loss
    print(np.array(accuracies_one).reshape([-1]).shape, np.array(accuracies_mem).reshape([-1]).shape)
    return np.array(accuracies_one).reshape([-1]), np.array(accuracies_mem).reshape([-1])

In [ ]:
 import torch 
 torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
configuration = {'x_updates': 1,  'theta_updates':1, 'factor': 1, 'x_lr': 0.0001,'th_lr':0.0001,\
            'device': torch.device("cuda" if torch.cuda.is_available() else "cpu"),\
            'batchsize':8, 'total_updates': 1000} 


import numpy as np
total_epoch=5000
print_it=50
total_runs=5
n_Tasks=2
acc_one = np.zeros((total_runs,((total_epoch//print_it-1)*n_Tasks)))
acc_m = np.zeros((total_runs,((total_epoch//print_it-1)*n_Tasks)))
name_label='MUTAG'
save_dir='MUTAG/'
for i in range(total_runs):
    acc_one[i,:], acc_m[i,:] =run(name_label, epochs=total_epoch, print_it=print_it, config=configuration)

    
    
    
accuracies_mem=np.mean(acc_m, axis=0)
accuracies_one=np.mean(acc_one, axis=0)
import matplotlib.pyplot as plt
plt.plot(accuracies_mem, label='memory accuracy')
plt.plot(accuracies_one, label='task accuracy')
np.savetxt(save_dir+name_label+'_acc_runs_5.csv', np.concatenate([accuracies_one, accuracies_mem],\
                                                                 axis=1), delimeter=',')

In [ ]:
plot_save(acc_m, acc_one, save_dir, name_label, total_epoch, print_it, total_runs, n_Tasks)